In [2]:
import os
from ultralytics import YOLO
import yaml
import json
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import cv2
from sklearn.metrics import confusion_matrix

# 소분류-대분류 매핑 정의
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items', 'Audio': 'Large Waste Items', 'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items', 'Main unit': 'Large Waste Items', 'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items', 'sewing machine': 'Large Waste Items', 'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items', 'Vacuum cleaner': 'Large Waste Items', 'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items', 'Sink': 'Large Waste Items', 'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items', 'Bed': 'Large Waste Items', 'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items', 'Cabinet': 'Large Waste Items', 'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items', 'Desk': 'Large Waste Items', 'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items', 'Display stand': 'Large Waste Items', 'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items', 'Small cabinet': 'Large Waste Items', 'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items', 'TV stand': 'Large Waste Items', 'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items', 'Air conditioner': 'Large Waste Items', 'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items', 'Electric rice cooker': 'Large Waste Items', 'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items', 'Heater': 'Large Waste Items', 'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items', 'refrigerator': 'Large Waste Items', 'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items', 'Washing machine': 'Large Waste Items', 'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items', 'Bedding items': 'Large Waste Items', 'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items', 'Clothes drying rack': 'Large Waste Items', 'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items', 'Earthenware jar': 'Large Waste Items', 'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items', 'lumber': 'Large Waste Items', 'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items', 'Piano': 'Large Waste Items', 'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items', 'Stroller': 'Large Waste Items', 'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items', 'audio cabinet': 'Large Waste Items', 'suitcase': 'Large Waste Items',
    
    'PP bag': 'PP bag',
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    'CleanNet': 'CleanNet'
}

class WasteDetectionTrainer:
    def __init__(self):
        # 대분류 클래스 및 색상 정의
        self.major_classes = ['Large Waste Items', 'PP bag', 'General Waste', 'CleanNet']
        self.class_to_idx = {name: idx for idx, name in enumerate(self.major_classes)}
        
        # 색상 정의 (대분류별로 다른 색상 사용)
        self.category_colors = {
            'Large Waste Items': (255, 0, 0),    # 빨강
            'PP bag': (0, 255, 0),               # 초록
            'General Waste': (0, 0, 255),        # 파랑
            'CleanNet': (255, 255, 0)            # 노랑
        }

    def verify_files(self, base_path):
        """파일 경로 검증"""
        required_files = {
            'Train Annotations': os.path.join(base_path, "train", "train_annotations_yolo.json"),
            'Valid Annotations': os.path.join(base_path, "valid", "valid_annotations_yolo.json"),
            'Test Annotations': os.path.join(base_path, "test", "test_annotations_yolo.json"),
            'Train Images': os.path.join(base_path, "train", "images"),
            'Valid Images': os.path.join(base_path, "valid", "images"),
            'Test Images': os.path.join(base_path, "test", "images")
        }
        
        for name, path in required_files.items():
            if not os.path.exists(path):
                raise FileNotFoundError(f"{name} not found at: {path}")
            print(f"Found {name} at: {path}")
        
        return required_files

    def setup_directory_structure(self, base_path):
        """디렉토리 구조 설정"""
        for split in ['train', 'valid', 'test']:
            labels_dir = os.path.join(base_path, split, 'labels')
            os.makedirs(labels_dir, exist_ok=True)

    def convert_annotations_with_mapping(self, json_path, output_dir):
        """소분류를 대분류로 매핑하여 YOLO 형식으로 변환"""
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            os.makedirs(output_dir, exist_ok=True)
            converted_count = 0
            processed_files = set()
            errors = []
            
            for image_name, annotations in data.items():
                txt_path = os.path.join(output_dir, Path(image_name).stem + '.txt')
                
                valid_annotations = []
                for ann in annotations:
                    try:
                        class_name = ann['class_name']
                        if class_name not in category_mapping:
                            errors.append(f"Unknown class: {class_name} in {image_name}")
                            continue
                            
                        major_category = category_mapping[class_name]
                        if major_category not in self.class_to_idx:
                            errors.append(f"Unknown major category: {major_category} in {image_name}")
                            continue
                            
                        class_idx = self.class_to_idx[major_category]
                        x_center, y_center, width, height = ann['bbox']
                        
                        if not all(0 <= coord <= 1 for coord in [x_center, y_center, width, height]):
                            errors.append(f"Invalid bbox coordinates in {image_name}: {ann['bbox']}")
                            continue
                            
                        valid_annotations.append((class_idx, x_center, y_center, width, height))
                        
                    except KeyError as e:
                        errors.append(f"Missing key {e} in annotation of {image_name}")
                    except Exception as e:
                        errors.append(f"Error processing annotation in {image_name}: {str(e)}")
                
                if valid_annotations:
                    with open(txt_path, 'w', encoding='utf-8') as f:
                        for ann in valid_annotations:
                            f.write(f"{ann[0]} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n")
                        converted_count += len(valid_annotations)
                        processed_files.add(image_name)
            
            print(f"Successfully converted {converted_count} annotations for {len(processed_files)} images")
            if errors:
                print("\nConversion errors occurred:")
                for error in errors[:10]:
                    print(error)
                if len(errors) > 10:
                    print(f"...and {len(errors) - 10} more errors")
                    
        except Exception as e:
            print(f"Error processing {json_path}: {str(e)}")
            raise

    def create_data_yaml(self, base_path, yaml_path):
        """YAML 설정 파일 생성"""
        data = {
            'path': base_path,
            'train': os.path.join('train', 'images'),
            'val': os.path.join('valid', 'images'),
            'test': os.path.join('test', 'images'),
            'nc': len(self.major_classes),
            'names': self.major_classes
        }
        
        with open(yaml_path, 'w', encoding='utf-8') as f:
            yaml.dump(data, f, default_flow_style=False, allow_unicode=True)
        
        print(f"\nCreated YAML file at: {yaml_path}")
        print(f"Number of classes: {len(self.major_classes)}")
        print("Classes in order:")
        for idx, name in enumerate(self.major_classes):
            print(f"{idx}: {name}")

    def create_augmentation_config(self):
        """데이터 증강 설정"""
        return {
            'hsv_h': 0.015,  # HSV 색조 증강
            'hsv_s': 0.7,    # HSV 채도 증강
            'hsv_v': 0.4,    # HSV 명도 증강
            'degrees': 10.0,  # 회전 각도
            'translate': 0.1, # 이미지 이동
            'scale': 0.5,    # 스케일 조정
            'fliplr': 0.5,   # 좌우 반전
            'mosaic': 1.0    # 모자이크 증강
        }

    def calculate_metrics(self, true_labels, pred_labels):
        """분류별 평가 지표 계산"""
        confusion_mat = confusion_matrix(true_labels, pred_labels, 
                                       labels=range(len(self.major_classes)))
        metrics = {}
        
        for i, class_name in enumerate(self.major_classes):
            tp = confusion_mat[i, i]
            fp = confusion_mat[:, i].sum() - tp
            fn = confusion_mat[i, :].sum() - tp
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            metrics[class_name] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'TP': int(tp),
                'FP': int(fp),
                'FN': int(fn)
            }
        
        return metrics, confusion_mat

    def plot_confusion_matrix(self, confusion_mat, save_path):
        """혼동 행렬 시각화"""
        plt.figure(figsize=(10, 8))
        sns.heatmap(confusion_mat, annot=True, fmt='d',
                    xticklabels=self.major_classes,
                    yticklabels=self.major_classes)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

    def draw_predictions(self, image_path, predictions, output_path):
        """예측 결과 시각화"""
        image = cv2.imread(image_path)
        for pred in predictions:
            class_id = int(pred['class'])
            class_name = self.major_classes[class_id]
            color = self.category_colors[class_name]
            
            bbox = pred['bbox']
            x1, y1, x2, y2 = [int(coord) for coord in bbox]
            
            # 박스 그리기
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            
            # 라벨 그리기
            label = f"{class_name} {pred['confidence']:.2f}"
            cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        cv2.imwrite(output_path, image)

    def train(self, base_path):
        """전체 학습 프로세스"""
        print("Starting training process...")
        
        # 파일 경로 검증
        required_files = self.verify_files(base_path)
        
        # 결과 디렉토리 생성
        results_dir = os.path.join(base_path, f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
        os.makedirs(results_dir, exist_ok=True)

        # 디렉토리 구조 설정
        self.setup_directory_structure(base_path)

        # YAML 파일 생성
        yaml_path = os.path.join(base_path, "data.yaml")
        self.create_data_yaml(base_path, yaml_path)

        # 소분류를 대분류로 변환하여 어노테이션 생성
        for split, json_path in {
            'train': required_files['Train Annotations'],
            'valid': required_files['Valid Annotations'],
            'test': required_files['Test Annotations']
        }.items():
            labels_dir = os.path.join(base_path, split, 'labels')
            print(f"\nConverting annotations for {split} set...")
            self.convert_annotations_with_mapping(json_path, labels_dir)

        # 모델 초기화 및 학습
        print("\nInitializing model...")
        model = YOLO('yolov8n.pt')
        augmentation_config = self.create_augmentation_config()

        try:
            # 모델 학습
            print("\nStarting model training...")
            results = model.train(
                data=yaml_path,
                epochs=100,
                imgsz=640,
                batch=8,
                name='waste_detection_model',
                project=results_dir,
                exist_ok=True,
                workers=4,
                patience=5,
                **augmentation_config
            )

            # 테스트 셋 평가
            print("\nEvaluating model on test set...")
            test_results = model.val(
                data=yaml_path,
                split='test'
            )

            # 테스트 이미지에 대한 예측 수행
            print("\nRunning inference on test images...")
            test_images_dir = os.path.join(base_path, "test", "images")
            visualization_dir = os.path.join(results_dir, "visualizations")
            os.makedirs(visualization_dir, exist_ok=True)

            predictions = model.predict(
                source=test_images_dir,
                save=True,
                save_txt=True,
                conf=0.25,
                project=results_dir,
                name='inference',
                boxes=True,
                labels=True
            )

            # 예측 결과 시각화
            print("\nVisualizing predictions...")
            for image_file in os.listdir(test_images_dir):
                if image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    image_path = os.path.join(test_images_dir, image_file)
                    output_path = os.path.join(visualization_dir, f"pred_{image_file}")
                    
                    image_predictions = model.predict(source=image_path, conf=0.25)[0]
                    pred_list = []
                    for box in image_predictions.boxes:
                        pred_list.append({
                            'class': box.cls.item(),
                            'confidence': box.conf.item(),
                            'bbox': box.xyxy[0].tolist()
                        })
                    self.draw_predictions(image_path, pred_list, output_path)

            # 평가 메트릭스 계산
            print("\nCalculating evaluation metrics...")
            true_labels = []
            pred_labels = []
            
            test_label_dir = os.path.join(base_path, "test", "labels")
            pred_label_dir = os.path.join(results_dir, "inference", "labels")
            
            for pred_file in os.listdir(pred_label_dir):
                true_file = os.path.join(test_label_dir, pred_file)
                pred_file_path = os.path.join(pred_label_dir, pred_file)
                
                if os.path.exists(true_file):
                    with open(true_file, 'r') as f:
                        for line in f:
                            true_labels.append(int(line.split()[0]))
                    
                    with open(pred_file_path, 'r') as f:
                        for line in f:
                            pred_labels.append(int(line.split()[0]))

            # 메트릭스 계산 및 저장
            metrics, conf_matrix = self.calculate_metrics(true_labels, pred_labels)
            
            # 결과 저장
            print("\nSaving evaluation results...")
            results_file = os.path.join(results_dir, 'evaluation_results.txt')
            with open(results_file, 'w') as f:
                f.write("Evaluation Results:\n\n")
                for class_name, class_metrics in metrics.items():
                    f.write(f"\n{class_name}:\n")
                    for metric_name, value in class_metrics.items():
                        f.write(f"{metric_name}: {value:.4f}\n")

            # 혼동 행렬 시각화 저장
            self.plot_confusion_matrix(conf_matrix, os.path.join(results_dir, 'confusion_matrix.png'))

            print(f"\nTraining and evaluation completed successfully!")
            print(f"Results saved in: {results_dir}")
            print(f"Visualizations saved in: {visualization_dir}")

        except Exception as e:
            print(f"\nError during training: {str(e)}")
            raise

def main():
    # 기본 경로 설정
    base_path = r"C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco"
    
    # 트레이너 초기화 및 학습 실행
    trainer = WasteDetectionTrainer()
    trainer.train(base_path)

if __name__ == "__main__":
    main()

Starting training process...
Found Train Annotations at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\train_annotations_yolo.json
Found Valid Annotations at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\valid_annotations_yolo.json
Found Test Annotations at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test\test_annotations_yolo.json
Found Train Images at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\images
Found Valid Images at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\images
Found Test Images at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test\images

Created YAML file at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\data.yaml
Number of classes: 4
Classes in order:
0: Large Waste Items
1: PP bag
2: General Waste
3: CleanNet

Converting annotations for train set...
Successfully converted 2795 annotations for 636 images

Conve

train: Scanning C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\labels... 636 images, 2 backgrounds, 0 corrupt: 100%|██████████| 638/638 [00:00<00:00, 1487.49it/s]

train: New cache created: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\labels.cache



val: Scanning C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\labels... 79 images, 0 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<00:00, 668.63it/s]


val: New cache created: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\labels.cache
Plotting labels to C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241124_155336\waste_detection_model\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241124_155336\waste_detection_model
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

val: Scanning C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test\labels... 79 images, 0 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<00:00, 1136.21it/s]

val: New cache created: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  3.85it/s]


                   all         79        428     0.0171     0.0233     0.0038   0.000755
     Large Waste Items         25         91     0.0265      0.033    0.00561    0.00105
                PP bag         11        102     0.0324     0.0294    0.00685    0.00123
         General Waste         56        130    0.00928     0.0308    0.00246   0.000558
              CleanNet         54        105          0          0   0.000296   0.000177
Speed: 0.6ms preprocess, 25.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241124_155336\waste_detection_model

Running inference on test images...
WARNING  'boxes' is deprecated and will be removed in in the future. Use 'show_boxes' instead.

Error during training: 'labels' is not a valid YOLO argument. Similar arguments are i.e. ['show_labels=True'].

    Arguments received: ['yolo', '--f=c:\\Users\\test\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3

SyntaxError: '[31m[1mlabels[0m' is not a valid YOLO argument. Similar arguments are i.e. ['show_labels=True'].

    Arguments received: ['yolo', '--f=c:\\Users\\test\\AppData\\Roaming\\jupyter\\runtime\\kernel-v36bdb5f63f9638af8525a96ab6f02b071430c51da.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'detect', 'segment', 'obb', 'pose', 'classify'}
                MODE (required) is one of {'export', 'val', 'train', 'track', 'predict', 'benchmark'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
     